# 태스크 3: 섀도우 테스트

이 노트북에서는 프로덕션 및 섀도우 모델 변형을 구성하고 테스트하여 섀도우 테스트 전략을 구현합니다. 프로덕션 및 섀도우 모델 변형의 결과를 검토하고 성능이 가장 우수한 모델을 파악합니다.

## 태스크 3.1: 환경 설정

필요한 라이브러리와 종속성을 설치합니다.

In [ ]:
%%capture
%matplotlib inline
from datetime import datetime, timedelta
import time
import os
import boto3
import re
import json
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.image_uris import retrieve
from sagemaker.session import production_variant
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

region = boto3.Session().region_name
role = get_execution_role()
sm_session = session.Session(boto3.Session())
sm = boto3.Session().client("sagemaker")
sm_runtime = boto3.Session().client("sagemaker-runtime")
cw = boto3.Session().client("cloudwatch")
s3 = boto3.Session().client("s3")

bucket = sm_session.default_bucket()
prefix = 'sagemaker/abalone'
data_capture_prefix = 'sagemaker/abalone/models'

## 태스크 3.2: 엔드포인트 만들기

이미 A/B 테스트를 사용하여 트래픽을 새 모델로 보냈습니다. AnyCompany는 비즈니스 요구 사항에 가장 적합한 A/B 테스트 또는 섀도우 테스트를 사용하는 데 관심이 있습니다. 배포 전략을 커밋하기 전에 다른 모델을 테스트하고 다른 배포 전략을 시도해 보십시오.

섀도우 테스트를 사용하면 프로덕션 엔드포인트로 보내는 트래픽을 중단하지 않고 새 모델로 트래픽을 보낼 수 있습니다. 섀도우 테스트는 성능이 좋지 않은 모델을 프로덕션에 배포할 때 발생할 수 있는 위험을 최소화하는 데 도움이 됩니다. 또한 추론에 동일한 데이터를 사용하여 두 모델을 더 현실적으로 비교하고 가동 중지 시간을 최소화할 수 있습니다. 

시작하려면 프로덕션 단계에 있는 현재 모델이 포함되어 있는 Amazon Simple Storage Service(S3) 버킷에 새 모델을 업로드합니다. 다음으로 프로덕션 엔드포인트를 만듭니다. 마지막으로 엔드포인트를 업데이트하여 새 모델을 섀도우 변형으로 추가합니다.

먼저 모델을 S3 버킷에 업로드합니다. 현재 프로덕션 단계의 모델은 *model_A.tar.gz*이고 새 모델은 *model_B.tar.gz*입니다.

In [ ]:
#upload-models
model_url = S3Uploader.upload(
    local_path="models/model_A.tar.gz", desired_s3_uri=f"s3://{bucket}/{prefix}"
)
model_url2 = S3Uploader.upload(
    local_path="models/model_B.tar.gz", desired_s3_uri=f"s3://{bucket}/{prefix}"
)
model_url, model_url2

다음으로 사전 훈련된 전복 모델에 대한 모델 정의를 만듭니다.

In [ ]:
#create-model-definitions
model_name = f"abalone-A-{datetime.now():%Y-%m-%d-%H-%M-%S}"
model_name2 = f"abalone-B-{datetime.now():%Y-%m-%d-%H-%M-%S}"
image_uri = retrieve("xgboost", boto3.Session().region_name, "1.5-1")
image_uri2 = retrieve("xgboost", boto3.Session().region_name, "1.5-1")

response = sm_session.create_model(
    name=model_name, role=role, container_defs={"Image": image_uri, "ModelDataUrl": model_url}
)

response_2 = sm_session.create_model(
    name=model_name2, role=role, container_defs={"Image": image_uri2, "ModelDataUrl": model_url2}
)

print(response)
print(response_2)

그런 다음 프로덕션 엔드포인트 구성을 만듭니다. *create_endpoint*를 사용하여 엔드포인트를 만들 때는 **EndpointName**과 **EndpointConfigName**을 포함해야 합니다. 

섀도우 테스트를 위한 엔드포인트 구성은 프로덕션 변형을 정의하는 것과 동일한 방식으로 정의됩니다. *create_endpoint*를 사용하면 **ProductionVariants**와 **ShadowVariants**를 사용하여 현재 프로덕션 모델과 새 섀도우 모델을 구분할 수 있습니다. 현재 프로덕션 단계에 있는 모델 A는 *ProductionVariant*로 설정됩니다. 섀도우 모드로 배포하려는 이전 실습의 모델인 모델 B는 실습의 뒷부분에서 *ShadowVariant*로 설정됩니다.

In [ ]:
#create-endpoint-configuration
endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = 'Abalone-Shadow-Testing-Endpoint',
    ProductionVariants=[
        {
            'ModelName':model_name,
            'InstanceType':'ml.m5.xlarge',
            'InitialInstanceCount':2,
            'VariantName':'Production-Model-A',
            'InitialVariantWeight':1
        }
    ]
)
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

마지막으로 엔드포인트 구성을 사용하여 엔드포인트를 만듭니다.

<i class="fas fa-sticky-note" style="color:#ff6633"></i> **참고:** 엔드포인트를 만드는 데 4~5분 정도 걸립니다.

In [ ]:
#create-endpoint
endpoint_name = f"Abalone-Shadow-{datetime.now():%Y-%m-%d-%H-%M-%S}"
endpoint_params = {'EndpointName': endpoint_name, 'EndpointConfigName': 'Abalone-Shadow-Testing-Endpoint'}

endpoint_response = sm.create_endpoint(EndpointName=endpoint_name, EndpointConfigName='Abalone-Shadow-Testing-Endpoint')
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=endpoint_response)

프로덕션 엔드포인트를 사용할 준비가 되었습니다. **invoke_endpoint**를 사용하여 엔드포인트로 트래픽을 보냅니다.

In [ ]:
#send-test-data
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")

with open("data/abalone_data_test.csv", "r") as f:
    for row in f:
        payload = row.rstrip("\n")
        sm_runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType="text/csv", Body=payload)
f.close()
print("Done!")

프로덕션에서 엔드포인트에 섀도우 모델을 추가하려면 두 단계가 필요합니다. 먼저 섀도우 프로덕션 변형을 포함하는 새 엔드포인트 구성을 만듭니다. 그런 다음 새 엔드포인트 구성을 사용하도록 엔드포인트를 업데이트합니다. 문제가 발생하는 경우 엔드포인트 구성을 업데이트하여 섀도우 모델을 롤백할 수 있습니다.

데이터 캡처를 사용하여 프로덕션 및 섀도우 변형 모두의 요청과 응답을 로깅합니다. 데이터 캡처 로그는 S3 버킷에 저장됩니다.

In [ ]:
#create-endpoint-configuration
endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = 'Abalone-Shadow-Testing-Endpoint-2',
    ProductionVariants = [
        {
            'ModelName':model_name,
            'InstanceType':'ml.m5.xlarge',
            'InitialInstanceCount':2,
            'VariantName':'Production-Model-A',
            'InitialVariantWeight':1
        }
    ],
    ShadowProductionVariants = [
        {
            'ModelName': model_name2,
            'InstanceType': 'ml.m5.xlarge',
            'InitialInstanceCount':2,
            'VariantName':'New-Model-B',
            'InitialVariantWeight':1,
        }
    ],
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': "s3://{}/{}".format(bucket, data_capture_prefix),
        'CaptureOptions': [{'CaptureMode': 'Input'}, {'CaptureMode': 'Output'}],
        'CaptureContentTypeHeader': {'CsvContentTypes': ['text/csv']}
    }
)
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

새 엔드포인트 구성을 사용하여 엔드포인트를 업데이트합니다. 그런 다음 엔드포인트가 다시 서비스를 제공할 때까지 기다립니다.

In [ ]:
#update-endpoint
endpoint_params = {'EndpointName': endpoint_name, 'EndpointConfigName': 'Abalone-Shadow-Testing-Endpoint-2'}

endpoint_response = sm.update_endpoint(EndpointName=endpoint_name, EndpointConfigName='Abalone-Shadow-Testing-Endpoint-2')
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

def wait_for_endpoint_update_complete(endpoint):
    """Helper function to wait for the completion of updating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Updating":
        print("Waiting for Endpoint to Update")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to update endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to update endpoint {endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {endpoint_response['EndpointArn']} successfully updated.")

wait_for_endpoint_update_complete(endpoint=endpoint_response)

셀이 완성되면 *'arn:aws:sagemaker:us-west-2:012345678910:endpoint/abalone-shadow-2025-01-01-01-01-00'*과 같은 엔드포인트 Amazon Resource Name(ARN)이 반환됩니다.

## 태스크 3.3: 호출 지표 평가

엔드포인트를 테스트하려면 배포된 모델을 호출하고 호출 지표를 평가합니다. 이전 태스크에서 만든 엔드포인트로 데이터를 보내 실시간으로 추론을 얻을 수 있습니다. 데이터 과학 팀이 새로운 데이터 집합을 받아서 당신에게 보냈습니다. 이 새 데이터를 사용하여 모델을 테스트합니다.

먼저 호출 샘플에 대한 테스트 데이터의 부분 집합을 가져옵니다.

In [ ]:
#import-data
shape=pd.read_csv("data/abalone_data_test.csv", header=0)
shape.sample(5)

그런 다음 **invoke_endpoint**를 사용하여 엔드포인트로 트래픽을 보냅니다.

<i class="fas fa-sticky-note" style="color:#ff6633"></i> **참고:** 엔드포인트 호출은 완료하는 데 1~2분 정도 걸립니다.

In [ ]:
#send-test-data
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")

with open("data/abalone_data_test.csv", "r") as f:
    for row in f:
        payload = row.rstrip("\n")
        sm_runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType="text/csv", Body=payload)
f.close()
print("Done!")

Amazon SageMaker는 Amazon CloudWatch의 각 변형에 대한 지연 시간과 호출과 같은 지표를 내보냅니다. 호출이 변형 간에 어떻게 분할되는지 보려면 CloudWatch에서 변형별 호출 수를 쿼리합니다.

In [ ]:
#get-cloudwatch-metrics
def get_invocation_metrics_for_endpoint_variant(endpoint_name, variant_name, start_time, end_time):
    metrics = cw.get_metric_statistics(
        Namespace="AWS/SageMaker",
        MetricName="Invocations",
        StartTime=start_time,
        EndTime=end_time,
        Period=60,
        Statistics=["Sum"],
        Dimensions=[
            {"Name": "EndpointName", "Value": endpoint_name},
            {"Name": "VariantName", "Value": variant_name},
        ],
    )
    return (
        pd.DataFrame(metrics["Datapoints"])
        .sort_values("Timestamp")
        .set_index("Timestamp")
        .drop("Unit", axis=1)
        .rename(columns={"Sum": variant_name})
    )


def plot_endpoint_metrics(start_time=None):
    start_time = start_time or datetime.now() - timedelta(minutes=60)
    end_time = datetime.now()
    metrics_variant1 = get_invocation_metrics_for_endpoint_variant(
        endpoint_name, 'Production-Model-A', start_time, end_time
    )
    metrics_variant2 = get_invocation_metrics_for_endpoint_variant(
        endpoint_name, 'New-Model-B', start_time, end_time
    )
    metrics_variants = metrics_variant1.join(metrics_variant2, how="outer")
    metrics_variants.plot()
    return metrics_variants

print("Waiting a minute for initial metric creation...")
time.sleep(120)  # The metrics and data capture need time to log and save all of the files
plot_endpoint_metrics()

표에서 프로덕션 모델인 모델 A의 호출 수를 확인할 수 있습니다. 또한 새 모델인 모델 B의 호출 수도 확인할 수 있습니다. 

이 차트는 일정 기간 동안의 모델 변형별 호출 수를 보여줍니다. 

트래픽이 모델 A에서만 시작되는 것을 확인할 수 있습니다. 이 동작은 유일한 모델 변형이 프로덕션 모델이었을 때 엔드포인트로 트래픽을 보냈기 때문에 발생합니다. 그런 다음 섀도우 모델이 추가되면 두 모델 모두 트래픽이 동일함을 확인할 수 있습니다. 선은 모든 포인트에서 서로 겹쳐집니다. 

## 태스크 3.4: 프로덕션 데이터를 사용한 모델 평가

모든 엔드포인트 입력 및 출력 데이터는 데이터 캡처를 사용하여 캡처됩니다. S3 버킷에서 로그 객체를 가져와서 데이터 캡처 로그에 액세스하고 각 이벤트를 확인할 수 있습니다. 데이터에 액세스할 때 입력이 *CSV*로 인코딩된다는 점에 유의하십시오. 엔드포인트로 보내지는 데이터는 *endpointInput*에 포함됩니다. 예측은 *endpointOutput*에 포함됩니다. 각 추론 이벤트에는 고유한 *eventId*가 있으므로 해당 *eventId*를 기반으로 프로덕션 및 섀도우 호출이 일치하도록 할 수 있습니다. 

모델을 평가하려면 S3 버킷에 저장되어 있는 로그를 사용합니다. 이러한 로그를 사용하면 단일 프로덕션 엔드포인트에서 프로덕션 모델과 섀도우 모델이 생성한 예측을 확인할 수 있습니다.

먼저 프로덕션 변형에서 캡처한 데이터의 로그 파일을 확인합니다.

In [ ]:
#view-production-variant-data
current_endpoint_capture_prefix = "{}/{}/{}".format(data_capture_prefix, endpoint_name, 'Production-Model-A')
result = s3.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
prod_var_capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]

def get_obj_body(obj_key):
    return s3.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

prod_var_capture_file = get_obj_body(prod_var_capture_files[-1])
print(json.dumps(json.loads(prod_var_capture_file.split('\n')[0]), indent=2))

그런 다음 프로덕션 변형 캡처 데이터를 pandas 데이터 프레임으로 변환합니다.

In [ ]:
#convert-production-data-to-pandas-dataframe
prod_input_list = []
for i in range(len(prod_var_capture_files)):
    prod_var_capture_file = get_obj_body(prod_var_capture_files[i])
    for i in range(len(prod_var_capture_file.split('\n'))):
        if not len(prod_var_capture_file.split('\n')[i]) == 0:
            prod_input = {}
            prod_input["input"] = json.loads(prod_var_capture_file.split('\n')[i])["captureData"]["endpointInput"]["data"]
            data = json.loads(json.loads(prod_var_capture_file.split('\n')[i])["captureData"]["endpointOutput"]["data"])
            prod_input["prod_output"] = data
            prod_input["eventId"] = json.loads(prod_var_capture_file.split('\n')[i])["eventMetadata"]["eventId"]
            prod_input_list.append(prod_input)

from pandas import json_normalize
prod_var_df = json_normalize(prod_input_list)
prod_var_df

다음으로 섀도우 변형에서 캡처한 데이터의 로그 파일을 확인합니다.

In [ ]:
#view-shadow-variant-data
current_endpoint_capture_prefix = "{}/{}/{}".format(data_capture_prefix, endpoint_name, 'New-Model-B')
result = s3.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
shadow_var_capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]

def get_obj_body(obj_key):
    return s3.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

shadow_var_capture_file = get_obj_body(shadow_var_capture_files[-1])
print(json.dumps(json.loads(shadow_var_capture_file.split('\n')[0]), indent=2))

그런 다음 섀도우 변형 캡처 데이터를 pandas 데이터 프레임으로 변환합니다.

In [ ]:
#convert-shadow-data-to-pandas-dataframe
shadow_input_list = []
for i in range(len(shadow_var_capture_files)):
    shadow_var_capture_file = get_obj_body(shadow_var_capture_files[i])
    for i in range(len(shadow_var_capture_file.split('\n'))):
        if not len(shadow_var_capture_file.split('\n')[i]) == 0:
            shadow_input = {}
            shadow_input["input"] = json.loads(shadow_var_capture_file.split('\n')[i])["captureData"]["endpointInput"]["data"]
            data = json.loads(json.loads(shadow_var_capture_file.split('\n')[i])["captureData"]["endpointOutput"]["data"])
            shadow_input["shadow_output"] = data
            shadow_input["eventId"] = json.loads(shadow_var_capture_file.split('\n')[i])["eventMetadata"]["eventId"]
            shadow_input_list.append(shadow_input)

shadow_var_df = json_normalize(shadow_input_list)
shadow_var_df

데이터 캡처는 배포 중에 많은 모델 변형과 추론 결과를 추적하는 데 도움이 됩니다. 방금 수집한 데이터를 사용하여 모델이 어떻게 비교되는지 확인할 수 있습니다.

프로덕션 변형과 섀도우 변형에서 예측된 전복 고리 값을 비교합니다. 원본 데이터 집합의 고리 표를 추가하여 실제 고리 수와 프로덕션 출력, 실제 고리 수와 섀도우 출력 간의 차이를 확인합니다.

완료되면 표에 다음이 포함됩니다.
- *eventId*에 대한 머리글
- 원본 *input*
- 프로덕션 모델 출력(*prod_output*)
- 섀도우 모델 출력(*shadow_output*)
- 전복의 실제 나이를 알려주는 원본 *labels*
- prod_output과 레이블의 차이(*prod_diff*)
- shadow_output과 레이블의 차이(*shadow_diff*)

In [ ]:
#compile-data-capture-with-labels
df_with_labels = pd.read_csv("data/abalone_data_new.csv", header=None)
test_labels = df_with_labels.iloc[:, 0]

final_df = pd.merge(prod_var_df, shadow_var_df, on='eventId', how='right')
final_df['labels'] =  test_labels
final_df = final_df.drop('input_y', axis=1)
final_df = final_df.assign(prod_diff=final_df['prod_output'] - final_df['labels'])
final_df = final_df.assign(shadow_diff=final_df['shadow_output'] - final_df['labels'])
final_df = final_df[['eventId','input_x','prod_output','shadow_output','labels','prod_diff','shadow_diff']]
final_df

각 예측의 분산을 검토하고 각 예측이 실제 값과 얼마나 차이가 나는지 확인합니다. 섀도우 배포 중에 각 추론 요청을 두 모델 모두에 보냈습니다. 따라서 결과에는 프로덕션 모델의 2,000개 예측과 섀도우 모델의 2,000개 예측이 포함됩니다. 예측을 평가하는 가장 간단한 방법은 예측과 실제 값 간의 차이의 절대값을 구한 다음 이러한 차이의 평균을 구하는 것입니다.

예를 들어 전복의 고리가 11개인데 프로덕션 예측이 9.3인 경우 이러한 값의 차이는 -1.7입니다. -1.7의 절대값은 1.7입니다. 모든 차이에 대한 절대값을 찾아 더한 다음 총합을 추론의 수로 나눕니다. 이러한 방식으로 절대값의 평균 차이를 구할 수 있습니다. 평균 차이가 적을수록 일반적으로 모델 예측이 더 정확함을 의미합니다.

In [ ]:
#calculate-average-absolute-difference
prod_diff_average = final_df.loc[:, 'prod_diff'].abs().mean()
shadow_diff_average = final_df.loc[:, 'shadow_diff'].abs().mean()

print("These are the averages of the absolute value of the difference between each model's inference results and the actual values (lower scores are more accurate):")
print("Production model variant: {}".format(prod_diff_average))
print("Shadow model variant: {}".format(shadow_diff_average))

이러한 경우 섀도우 모델은 프로덕션 모델 변형보다 평균 차이가 더 큽니다. 해당 결과는 섀도우 모델이 현재 프로덕션 모델만큼 실제 고리 수를 잘 예측하지 못할 가능성이 크다는 것을 나타냅니다.

데이터 캡처 로그에서 수집한 데이터의 차트를 만들어 결과를 더 자세히 확인할 수 있습니다. 프로덕션 모델과 섀도우 모델의 고리 수의 평균 차이를 그래프로 작성합니다. 그래프는 각 고리 수에 대해 각 모델이 올바른 값을 예측하는 데 얼마나 근접했는지를 보여줍니다.

In [ ]:
#graph-model-differences-from-labels
final_df['prod_diff'] = final_df['prod_diff'].abs()
final_df['shadow_diff'] = final_df['shadow_diff'].abs()
grouped_df = final_df.groupby(['labels']).agg(prod_diff_average=('prod_diff', 'mean'), shadow_diff_average=('shadow_diff' , 'mean'))
grouped_df = grouped_df.reset_index()
grouped_df = grouped_df.melt(id_vars=['labels'], value_vars=['prod_diff_average', 'shadow_diff_average'])

sb.barplot(x=grouped_df.labels,
           y=grouped_df.value,
           hue=grouped_df.variable,
           data=grouped_df).set(title='Difference in Prediction Scores by Number of Rings', xlabel='Number of Rings', ylabel='Absolute Difference in Predicted to Actual Rings')

이 차트는 프로덕션 모델과 섀도우 모델의 각 고리 값에 대한 차이가 비슷함을 보여줍니다. 프로덕션 모델은 일부 고리 값의 분산이 더 낮고 섀도우 모델은 기타 고리 값의 분산이 더 낮습니다.

예측 값의 목록을 수집했습니다. 이제 각 변형의 R 제곱 점수(R2 점수), 평균 절대 오차(MAE), 평균 제곱근 오차(RMSE) 지표를 평가할 수 있습니다. 이러한 지표는 새 모델이 현재 프로덕션 단계에 있는 모델보다 개선되었는지 여부를 판단하는 데 도움이 됩니다.

In [ ]:
#evaluate-model-metrics
labels = final_df['labels']
prod_preds = final_df['prod_output']
shadow_preds = final_df['shadow_output']

# Calculate R2
prod_score = r2_score(labels, prod_preds)
shadow_score = r2_score(labels, shadow_preds)
print("The R2 score of the production model is {}".format(round(prod_score, 2)))
print("The R2 score of the shadow model is {}\n".format(round(shadow_score, 2)))

# Calculate MAE
prod_score = mean_absolute_error(labels, prod_preds)
shadow_score = mean_absolute_error(labels, shadow_preds)
print("The Mean Absolute Error of the production model is {}".format(round(prod_score, 2)))
print("The Mean Absolute Error of the shadow model is {}\n".format(round(shadow_score, 2)))

# Calculate RMSE
prod_score = np.sqrt(mean_absolute_error(labels, prod_preds))
shadow_score = np.sqrt(mean_absolute_error(labels, shadow_preds))
print("The Root Mean Squared Error of the production model is {}".format(round(prod_score, 2)))
print("The Root Mean Squared Error of the shadow model is {}\n".format(round(shadow_score, 2)))

섀도우 모델의 성능이 현재 프로덕션 모델만큼 좋지 않습니다. R2 점수는 더 낮고, MAE는 더 높으며, RMSE도 더 높습니다. R2 점수가 낮을수록 섀도우 모델이 현재 프로덕션 모델에 비해 개선되지 않았다고 확신할 수 있습니다. 팀에서는 이러한 결과를 개선하기 위해 다시 모델을 테스트하고 튜닝하고 있습니다.

프로덕션 모델 변형과 섀도우 모델 변형을 구성하고 테스트하여 섀도우 테스트 전략을 성공적으로 구현했습니다. 프로덕션 및 섀도우 모델 변형의 결과를 검토하고 성능이 가장 우수한 모델을 파악했습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 **Conclusion**을 계속 진행합니다.